## 연도별 히트곡을 가장 많이 제작한 기획사/작곡가/작사가 TOP5 추출

### 스크래핑 대상
- 멜론 연도별 TOP 100
  - 2020 ~ 2024 : 연간차트 스크래핑
  - 2025 : 추후 월간 순위 취합하여 산출 

### 스크래핑 정보
- 연도, 순위, 노래ID, 앨범ID

In [1]:
# 스크래핑에 필요한 라이브러리를 불러와봅시다.

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re

In [2]:
options = Options()
options.add_argument(
    "--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
    "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36"
)

driver = webdriver.Chrome(options=options, service=Service(ChromeDriverManager().install()))
driver.get("https://www.melon.com/chart/index.htm")
wait = WebDriverWait(driver, 10)

# 차트 파인더 클릭
button = driver.find_element(By.CLASS_NAME, 'btn_chart_f')
ActionChains(driver).click(button).perform()

# 차트선택 : 연도차트 선택
button = driver.find_element(By.CLASS_NAME, 'tab03')
ActionChains(driver).click(button).perform()

# ========== 연대 선택 루프 ==========
decade_labels = wait.until(EC.presence_of_all_elements_located(
    (By.CSS_SELECTOR, "#d_chart_search > div > div > div.box_chic.nth1.view > div.list_value > ul > li > span > label")
))

TO_FIND_DECADE = ['2020년대']
for label in decade_labels:
    if label.text in TO_FIND_DECADE:
        label.click()

        # ========== 연도 선택 루프 ==========
        year_labels = wait.until(EC.presence_of_all_elements_located(
            (By.CSS_SELECTOR, "#d_chart_search > div > div > div.box_chic.nth2.view > div.list_value > ul > li > span > label")
        ))
        TO_FIND_YEAR = ['2024년', '2023년', '2022년', '2021년', '2020년']
        YEARLY_CHARTS = []
        for label in year_labels:
            # TODO. 향후 if문 제거예정
            if label.text in TO_FIND_YEAR:
                label.click()
    
                # 국내종합 선택 (안정적으로 대기 후 클릭)
                style = wait.until(EC.element_to_be_clickable((
                    By.CSS_SELECTOR,
                    '#d_chart_search > div > div > div.box_chic.last.view > div.list_value > ul > li:nth-child(2) > span > label'
                )))
                style.click()
    
                # 검색 버튼 클릭
                search = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'even_span')))
                search.click()
    
                time.sleep(1) 
    
                # table select
                tbody_obj = wait.until(EC.presence_of_element_located((By.ID, "chartListObj")))
                tr_list = tbody_obj.find_elements(By.TAG_NAME, "tr")
                rank = 0
                for tr in tr_list:
                    rank += 1
                    print(rank)
                    a_tags = tr.find_elements(By.TAG_NAME, "a")

                    # ID 초기화
                    song_id = None
                    album_id = None
                    
                    # 정규 표현식 컴파일
                    # 1. 노래 ID 패턴: goSongDetail('숫자') 추출
                    song_pattern = re.compile(r"goSongDetail\('(\d+)'\)")
                    # 2. 앨범 ID 패턴: goAlbumDetail('숫자') 추출
                    album_pattern = re.compile(r"goAlbumDetail\('(\d+)'\)")
                    
                    # a_tags를 순회하며 href값 찾기
                    for a_tag in a_tags:
                        # 이미 두 ID를 모두 찾았다면 루프를 종료
                        if song_id is not None and album_id is not None:
                            break
                            
                        # 각 <a> 태그에서 href 속성 값을 가져오기
                        current_href = a_tag.get_attribute('href')
                        
                        # href 속성이 존재하는 경우에만 처리
                        if current_href:
                            # 1. 노래 ID 매칭 시도
                            if song_id is None:
                                match_song = song_pattern.search(current_href)
                                if match_song:
                                    song_id = match_song.group(1)
                            
                            # 2. 앨범 ID 매칭 시도
                            if album_id is None:
                                match_album = album_pattern.search(current_href)
                                if match_album:
                                    album_id = match_album.group(1)

                    if song_id and album_id:
                        print("✅ 노래 ID와 앨범 ID를 모두 성공적으로 찾았습니다.")
                        print(f"노래 ID: {song_id}")
                        print(f"앨범 ID: {album_id}")
                    elif song_id:
                        print(f"⚠️ 앨범 ID는 찾지 못했지만, 노래 ID: {song_id}를 찾았습니다.")
                    elif album_id:
                        print(f"⚠️ 노래 ID는 찾지 못했지만, 앨범 ID: {album_id}를 찾았습니다.")
                    else:
                        print("❌ 경고: 노래 ID와 앨범 ID 패턴을 모두 찾을 수 없습니다.")
                    
                    # 수집한 정보 yearly_charts에 추가
                    data = {
                        "year" : label.text[:-1],
                        "rank" : rank,
                        "song_id" : song_id,
                        "album_id" : album_id,
                        "source" : "melon"                        
                    }
                    YEARLY_CHARTS.append(data)


driver.quit()
                

1
✅ 노래 ID와 앨범 ID를 모두 성공적으로 찾았습니다.
노래 ID: 37140709
앨범 ID: 11391902
2
✅ 노래 ID와 앨범 ID를 모두 성공적으로 찾았습니다.
노래 ID: 37138469
앨범 ID: 11402655
3
✅ 노래 ID와 앨범 ID를 모두 성공적으로 찾았습니다.
노래 ID: 37524037
앨범 ID: 11487023
4
✅ 노래 ID와 앨범 ID를 모두 성공적으로 찾았습니다.
노래 ID: 37145732
앨범 ID: 11404142
5
✅ 노래 ID와 앨범 ID를 모두 성공적으로 찾았습니다.
노래 ID: 31927275
앨범 ID: 10307346
6
✅ 노래 ID와 앨범 ID를 모두 성공적으로 찾았습니다.
노래 ID: 37023625
앨범 ID: 11376816
7
✅ 노래 ID와 앨범 ID를 모두 성공적으로 찾았습니다.
노래 ID: 30232719
앨범 ID: 10036099
8
✅ 노래 ID와 앨범 ID를 모두 성공적으로 찾았습니다.
노래 ID: 37347911
앨범 ID: 11450069
9
✅ 노래 ID와 앨범 ID를 모두 성공적으로 찾았습니다.
노래 ID: 37228861
앨범 ID: 11421941
10
✅ 노래 ID와 앨범 ID를 모두 성공적으로 찾았습니다.
노래 ID: 37373234
앨범 ID: 11454681
11
✅ 노래 ID와 앨범 ID를 모두 성공적으로 찾았습니다.
노래 ID: 37003911
앨범 ID: 11372896
12
✅ 노래 ID와 앨범 ID를 모두 성공적으로 찾았습니다.
노래 ID: 37053556
앨범 ID: 11382987
13
✅ 노래 ID와 앨범 ID를 모두 성공적으로 찾았습니다.
노래 ID: 36699489
앨범 ID: 11305759
14
✅ 노래 ID와 앨범 ID를 모두 성공적으로 찾았습니다.
노래 ID: 37390939
앨범 ID: 11459325
15
✅ 노래 ID와 앨범 ID를 모두 성공적으로 찾았습니다.
노래 ID: 37563682
앨범 ID: 11475749
16
✅

In [3]:
print(len(YEARLY_CHARTS))
YEARLY_CHARTS[0]

500


{'year': '2024',
 'rank': 1,
 'song_id': '37140709',
 'album_id': '11391902',
 'source': 'melon'}

In [6]:
# csv 저장
import csv
import os

# 저장할 파일명
csv_dir = os.path.join(os.getcwd() , 'csv')
csv_file_path = os.path.join(csv_dir, 'melon_yearly_top100.csv')

# 딕셔너리의 키를 필드 이름(헤더)으로 사용합니다.
fieldnames = YEARLY_CHARTS[0].keys()

try:
    with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
        # DictWriter 객체 생성
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        # 헤더(필드 이름) 쓰기
        writer.writeheader()
        
        # 데이터를 한 줄씩 쓰기
        writer.writerows(YEARLY_CHARTS)
        
    print(f"✅ 성공적으로 저장")

except Exception as e:
    print(f"❌ 파일 저장 중 오류 발생: {e}")

✅ 성공적으로 저장
